In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import bioframe as bf
import pandas as pd
import numpy as np
from upsetplot import from_indicators
from gimmemotifs.motif import Motif,read_motifs

In [ ]:
df_ES2i = pd.read_excel('~/Supplemental_Table_S5.xlsx', header = 3)
df_ESS = pd.read_excel('~/Supplemental_Table_S6.xlsx', header = 3)
df_NPC = pd.read_excel('~/Supplemental_Table_S7.xlsx', header = 3)

Creation of plot to compare all factors with one another to determine which variables are important for motif discovery. 

In [ ]:
def var_analysis(list_of_dfs):
    for df in list_of_dfs:
        score = pd.DataFrame(df, columns= ['score'])
        pvalue = pd.DataFrame(df, columns= ['pValue'])
        signalvalue = pd.DataFrame(df, columns= ['signalValue'])
        qvalue = pd.DataFrame(df, columns= ['qValue'])
        peak = pd.DataFrame(df, columns= ['qValue'])
    fig, ax = plt.subplots(2, 3, constrained_layout=True)
    ax[0][0].plot(score, pvalue, label = "score,pvalue")
    ax[0][0].set(title = "", xlabel='Score', ylabel='pValue')

    ax[0][1].plot(score, qvalue, label = "score,qvalue")
    ax[0][1].set(title = "", xlabel='Score', ylabel='qValue')

    ax[0][2].plot(score, peak, label = "score,peak")
    ax[0][2].set(title = "", xlabel='Score', ylabel='peak')

    ax[1][0].plot(score, signalvalue, label = "score,signalValue")
    ax[1][0].set(title = "", xlabel='Score', ylabel='signalValue')

    ax[1][1].plot(signalvalue, pvalue, label = "score,pValue")
    ax[1][1].set(title = "", xlabel='signalValue', ylabel='pValue')

    ax[1][2].plot(signalvalue, qvalue, label = "score,qValue")
    ax[1][2].set(title = "", xlabel='signalValue', ylabel='qValue')

    plt.show()
    return 0

list_of_dfs = [df_ES2i, df_ESS, df_NPC]
var_analysis(list_of_dfs)



Function and call to identify overlaps between cell types. 

In [ ]:
def make_indicators(list_of_dfs):
    name_arr = ['ES2i', 'ESSerum', 'NPC']
    num_df = len(list_of_dfs)
    df_tot = pd.concat(list_of_dfs) 
    print (df_tot)
    i = 0
    j = 0 
    for df in list_of_dfs:
        #if any given interval in df_tot overlaps any interval in df
        df_tot = bf.count_overlaps(df_tot, df,
                                             cols1=['#chr','start','end'], cols2=['#chr','start','end'])
        df_tot['count'] = df_tot['count'].astype(bool)
        df_tot.rename(columns={'count':name_arr[i]},inplace=True)
        i = i + 1
    return df_tot
#df_es_2i1_ctcf,df_es_serum_ctcf,df_npc1_ctcf
list_of_dfs = [df_ES2i,df_ESS,df_NPC]
overlap_indicators = make_indicators(list_of_dfs)
display (overlap_indicators)

In [ ]:
es2i_ess= overlap_indicators[(overlap_indicators['ES2i'] == True) & (overlap_indicators['ESSerum'] == True) & (overlap_indicators['NPC'] == False)]
display(es2i_ess)


npc_ess= overlap_indicators[(overlap_indicators['NPC'] == True) & (overlap_indicators['ESSerum'] == True) & (overlap_indicators['ES2i'] == False)]
display(npc_ess)

es2i_npc = overlap_indicators[(overlap_indicators['ES2i'] == True) & (overlap_indicators['NPC'] == True) & (overlap_indicators['ESSerum'] == False)]
display(es2i_npc)

tot = overlap_indicators[(overlap_indicators['ES2i'] == True) & (overlap_indicators['NPC'] == True) & (overlap_indicators['ESSerum'] == True)]
display(tot)

Sorting overlaps by top thousand outputs based on pValue to maximize quality of motif discovery and reduce noise. This function also cleans data as necessary. 

In [ ]:
def top_thousand_sort(df_edit):
    df_edit.rename(columns={'#chr':'chrom'},inplace=True)
    df_annotated = bf.cluster(df_edit, min_dist=0)
    display(df_annotated)
    df_annotated = df_annotated.groupby(['cluster']).max()
    df_annotated['start'] = df_annotated["cluster_start"]
    df_annotated['end'] = df_annotated["cluster_end"]
    df_final = df_annotated.iloc[:,0:9]
    display(df_final)
    df_final_thousand = df_final.sort_values(by=['pValue'], ascending=False)
    display(df_final_thousand)
    df_final_thousand = df_final_thousand.head(1000)
    display(df_final_thousand)
    df_final_thousand.rename(columns={'chrom':'#chr'},inplace=True)
    return (df_final_thousand)

In [ ]:
tes2i_ess = top_thousand_sort(es2i_ess)
tes2i_npc = top_thousand_sort(es2i_npc)
tnpc_ess = top_thousand_sort(npc_ess)
ttot = top_thousand_sort(tot)

In [ ]:
tes2i_ess.to_csv('~/es2i_ess.csv')
tes2i_npc.to_csv('~/es2i_npc.csv') 
tnpc_ess.to_csv('~/npc_ess.csv') 
ttot.to_csv('~/tot.csv')

Output of Motif Discovery from Gimmemotifs

In [ ]:
pfm_motif_es2i_npc = read_motifs("/home1/chavali/gimme.es2inpc/gimme.denovo.pfm")
display(pfm_motif_es2i_npc)

pfm_motif_es2i_ess = read_motifs("/home1/chavali/gimme.CTCFes2iess/gimme.denovo.pfm")
display(pfm_motif_es2i_ess)

pfm_motif_npc_ess = read_motifs("/home1/chavali/gimme.npc_ess/gimme.denovo.pfm")
display(pfm_motif_npc_ess)

pfm_motif_tot = read_motifs("/home1/chavali/gimme.tot/gimme.denovo.pfm")
display(pfm_motif_tot)

pfm_motif_es2i = read_motifs("/home1/chavali/gimme.CTCFes2i/gimme.denovo.pfm")
display(pfm_motif_es2i)

pfm_motif_npc = read_motifs("/home1/chavali/gimme.CTCFnpc/gimme.denovo.pfm")
display(pfm_motif_npc)

pfm_motif_ess = read_motifs("/home1/chavali/gimme.CTCFess/gimme.denovo.pfm")
display(pfm_motif_ess)

#Can already extrapolate sequence from PFM. 

Homer-Score Analysis of DeNovo Motifs 

In [ ]:
#import as pfm and then compare as opposed to copy pasting to ensure reproducibility

motifs = [
    motif_from_consensus("TGCCmyCTnsTGGC"),
    motif_from_consensus("rATCTGnTGCCmTCTTCT"),
    motif_from_consensus("myrGCGCCCyCTTCTGGC"),
    motif_from_consensus("CCAsnAGrkGGCA"),
    motif_from_consensus("GrkGGCAsyr"),
    motif_from_consensus("GyCACy"),
    motif_from_consensus("CAnTGnCwsC"),
    motif_from_consensus("rCTsCnAn"),
    motif_from_consensus("sCAsTr"),
    motif_from_consensus("nnGCCAsyAGrGGGCGCy"),
    motif_from_consensus("GsGGCGn"),
    motif_from_consensus("ynGCCAsyAGrGGGCrCy"),
    motif_from_consensus("nrGCGC"),
    motif_from_consensus("nnrGyGCCCyCTrsTGGyCn"),
    motif_from_consensus("GCGCCCyCTn"),
    motif_from_consensus("GCGCys"),
    motif_from_consensus("yGCCCy"),
    motif_from_consensus("ynrCCAsyAGrkGGCrsynn"),
    ###Only motif from JASPAR
    ##import using PFM and compare
]

mc = MotifComparer()
results = mc.get_closest_match(motifs, dbmotifs=read_motifs("HOMER"), metric="seqcor")

# Load motifs
db = read_motifs("HOMER", as_dict=True)

for motif in motifs:
    match, scores = results[motif.id]
    print("{}: {} - {:.3f}".format(motif.id, match, scores[0]))
    dbmotif = db[match]
    orient = scores[2]
    if orient == -1:
        dbmotif = dbmotif.rc()
    padm, padd = 0, 0
    if scores[1] < 0:
        padm = -scores[1]
    elif scores[1] > 0:
        padd = scores[1]
    print(" " * padm + motif.to_consensus())
    print(" " * padd + dbmotif.to_consensus())
    print()